# Get PC Information from Local Cure PCs
This script runs the following processes: Lookup local cure machine's pcid, ip address, and CPC software version

## Setup

### Import Libraries

In [ ]:
print("Initializing Python script...\n")

print("Importing libraries...\n")
# Standard Library Imports
import os
import socket
import sys
from datetime import datetime
from pathlib import Path

# Third-Party Imports
from natsort import index_natsorted, order_by_index
import pandas as pd
from sqlalchemy import create_engine, inspect

# Project (Local and Global) Imports
base = Path().resolve().parents[3] # monorepo root
sys.path.insert(0, str(base))
print(base)

from shared_config.src.shared_config.sql import PROD_SERVER as server, PYRO_SERVER as pyro_server, PYRO_DATABASE as database
from shared_utils.src.shared_utils.sql import get_engine, read_table, write_to_sql

### Settings
#### Define Notebook Settings

In [ ]:
# Toggle max rows and max columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Set CPC Variables

In [ ]:
source = r"C\Program Files\CPC Client"
client = "CPCClient.exe"

desired_version = "2023-03-13"

### Connect to Pyrometry Database on Production SQL Server

In [ ]:
print(f'Connecting to {server} server {database} database...\n')

engine = get_engine(server, database)

equip_pcs = read_table(engine, 'SELECT * FROM [Equipment PCs]')

inspector = inspect(engine)
inspector.get_table_names()

## Functions

### Get IP Address with PCID

In [ ]:
def get_ip(pcid):
    try:
        return socket.gethostbyname(pcid)
    except:
        return None

### Get PCID with IP Address

In [ ]:
def get_pcid(ip_address):
    try:
        hostname = socket.gethostbyaddr(ip_address)[0]
        pcid = hostname.split('.')[0].upper()
        return pcid
    except:
        return None

### Get CPC Version with IP Address or PCID

In [ ]:
def get_cpc(ip, pcid, source, client):
    try:
        client_cpc = os.path.join('\\' + '\\' + str(ip), source, client)

        if not os.path.exists(client_cpc):
            client_cpc = os.path.join('\\' + '\\' + str(pcid), source, client)
            
        if os.path.exists(client_cpc):
            client_date = datetime.fromtimestamp(os.path.getmtime(client_cpc)).strftime('%Y-%m-%d')
            return str(client_date)
        return None
    except Exception:
        return None

## Run Code

### Extract Info from Local PCs

In [ ]:
# IP Addresses
equip_pcs['New_IP_Address'] = equip_pcs['PCID'].apply(lambda pcid: get_ip(pcid))
equip_pcs['IP_Address'] = equip_pcs.apply(
    lambda row: row['New_IP_Address'] if row['New_IP_Address'] is not None else row['IP_Address'], axis=1
) # If New IP Address is None, previous IP Address is used

# PCIDs
equip_pcs['New_PCID'] = equip_pcs['IP_Address'].apply(lambda ip: get_pcid(ip))
equip_pcs['PCID'] = equip_pcs.apply(
    lambda row: row['New_PCID'] if row['New_PCID'] is not None else row['PCID'], axis=1
) # If New PCID is None, previous PCID is used

# CPC Versions
equip_pcs['New_CPC_Version'] = equip_pcs.apply(
    lambda row: get_cpc(row['IP_Address'], row['PCID'], source, client), axis=1
)
equip_pcs['CPC_Version'] = equip_pcs.apply(
    lambda row: row['New_CPC_Version'] if row['New_CPC_Version'] is not None else row['CPC_Version'], axis=1
) # If New CPC Version is None, previous CPC Version is used

## Clean Data

In [ ]:
equip_pcs = equip_pcs.drop(columns=['New_IP_Address', 'New_PCID', 'New_CPC_Version'])
equip_pcs['Database_Name'] = equip_pcs['Alt_Name'].apply(lambda alt_name: f"CPC_{alt_name.replace(' ', '')}")

indexer = index_natsorted(equip_pcs['PC'])
equip_pcs = equip_pcs.reindex(order_by_index(equip_pcs.index, indexer))
equip_pcs = equip_pcs.reset_index(drop=True)

## Explore Data

In [ ]:
equip_pcs[equip_pcs['Site'] == 'HTS']

## Output

In [ ]:
write_to_sql(equip_pcs, 'Equipment PCs', engine)